In [24]:
# import pandas as pd
# import datetime as dt
# from lifelines import CoxPHFitter
# import numpy as np
# import json
# import pickle

# start = pd.to_datetime('today')
# print(start)

# df = pd.read_csv('C:\Users\a\Documents\Projects\doh_app\static\DOHMH_New_York_City_Restaurant_Inspection_Results (3).csv')

# df.columns = [x.lower() for x in df.columns]
# df.columns = [x.replace(' ','_') for x in df.columns]


# df = df[(df['inspection_type'].str.contains('Cycle')) | 
# (df['inspection_type'].str.contains('Pre')) |
#         (pd.isna(df['inspection_type']))]

# df['inspection_date'] = df['inspection_date'].astype('datetime64[ns]')
# df['record_date'] = df['record_date'].astype('datetime64[ns]')
# df['zipcode'] = df['zipcode'].astype(str).replace(r'\.0', '', regex=True)
# df = df[['camis', 'dba', 'building', 'street', 'boro', 'inspection_date', 'action', 'score','inspection_type','record_date','violation_description',
#         'latitude','longitude','cuisine_description']]



# # df['inspection_id'] = df['camis'].astype(str) + df['inspection_date'].astype(str)
# df['address'] = df['building'] + ' ' + df['street'] + ', ' + df['boro']

# # df['rest_label'] = df['rest_label'].where(pd.notnull(df['rest_label']), df['dba'])
# df['dba'] = df['dba'].astype(str).apply(lambda x: ' '.join(x.split()).title())
# df['address'] = df['address'].astype(str).apply(lambda x: ' '.join(x.split()).title())
# df['address'] = df['address'].str.replace('\s{2,}', ' ', regex=True)


# df.drop(columns=['building','street','boro'], inplace=True)


# small = df['dba'].apply(lambda x: x.lower())
# small = small.str.replace('[^a-zA-Z0-9 ]', '', regex=True)
# df['small'] = small

# df = df.sort_values(['camis','inspection_date'], ascending=True)

# # df.sort_values(['camis','inspection_date'], inplace=True)
# # df.drop_duplicates(subset='inspection_id',inplace=True)

# df = df[df['inspection_date']!='1900-01-01']
# df.reset_index(inplace=True, drop=True)

# df['next_grade'] = df['inspection_date'].shift(-1)

# df.loc[df.drop_duplicates("camis", keep='last').index,'next_grade'] = pd.NaT

# df['time_til'] = df['next_grade'] - df['inspection_date']

# df['time_til'] = df['time_til'].where(pd.notnull(df['time_til']), (df['record_date'] - df['inspection_date']))

# df['event'] = df['next_grade'].apply(lambda x: 1 if pd.notnull(x) else 0)

# df['time_til'] = df['time_til'].dt.days

# df = df[df['time_til'] < 800]
# df.reset_index(drop=True, inplace=True)

# inspection_bin = []

# for i in range(len(df)):
#     current_type = df.loc[i,'inspection_type']
#     current_score = df.loc[i,'score']
#     current_camis = df.loc[i,'camis']
#     current_action = df.loc[i,'action']
#     current_time = df.loc[i,'time_til']
#     current_event = df.loc[i,'event']

#     if (df.loc[i,'inspection_date'] < pd.Timestamp('2020-03-17')) & (current_event == 0):
#         inspection_bin.append('covid')

#     elif (df.loc[i,'inspection_date'] < pd.Timestamp('2020-03-17')) & (df.loc[i,'next_grade'] > pd.Timestamp('2021-07-19')):
#         inspection_bin.append('covid')
    
#     elif 'losed' in current_action:
#         inspection_bin.append('was_closed')
        
#     elif 're-open' in current_action:
#         inspection_bin.append('re-opened')
        
#     elif 'Cycle' in current_type:
#         if 'Initial' in current_type:
#             if (current_score < 14) & (current_event == 1) & (current_time < 300):
#                 inspection_bin.append('cyc_init_1')
#             elif current_score < 14: # This event shouldn't be able to occur
#                 inspection_bin.append('cyc_init_0')
#             elif current_score > 13:
#                 inspection_bin.append('cyc_init_1')
                
#         elif 'Re-' in current_type:
#             previous_score = df.loc[i-1,'score']
#             previous_camis = df.loc[i-1,'camis']
            
#             if current_camis == previous_camis:
#                 if previous_score < 14:
#                     inspection_bin.append('cyc_re_0')
#                 elif previous_score > 13 and previous_score < 28:
#                     inspection_bin.append('cyc_re_1')
#                 elif previous_score > 27:
#                     inspection_bin.append('cyc_re_2')
#             else:
#                 inspection_bin.append('missing_prior_cycle')
#         else:
#             inspection_bin.append('other_cycle')
            
            
#     elif 'Pre-' in current_type:
#         if 'Initial' in current_type:
#             if (current_score < 14) & (current_event == 1) & (current_time < 300):
#                 inspection_bin.append('pre_init_1')
#             elif current_score < 14:
#                 inspection_bin.append('pre_init_0')
#             elif current_score > 13:
#                 inspection_bin.append('pre_init_1')
                
#         elif 'Re-' in current_type:
#             previous_camis = df.loc[i-1,'camis']
#             previous_score = df.loc[i-1,'score']
            
#             if current_camis == previous_camis:
#                 if previous_score < 14:
#                     inspection_bin.append('pre_re_0')
#                 elif previous_score > 13 and previous_score < 28:
#                     inspection_bin.append('pre_re_1')
#                 elif previous_score > 27:
#                     inspection_bin.append('pre_re_2')
#             else:
#                 inspection_bin.append('missing_prior_pre')
#         else:
#             inspection_bin.append('other_pre')
#     else:
#         inspection_bin.append('other')
        

# inspection_bin = pd.Series(inspection_bin)

# df['inspection_bin'] = inspection_bin


# # df.to_csv('parsed_df.csv')

# model_args = df.drop_duplicates('camis', keep="last")[['time_til','event','inspection_bin','camis']]
# model_args.reset_index(drop=True, inplace=True)

# # result = model_args.to_json(orient='records')
# # parsed = json.loads(result)

# # with open("lookup.json", "w") as outfile:
# #     json.dump(parsed, outfile)


# # cph = CoxPHFitter()
# # cph.fit(df[['time_til','event','inspection_bin']], duration_col='time_til', event_col='event', strata=['inspection_bin'])
# # filename = 'todays_model.sav'
# # pickle.dump(cph, open(filename, 'wb'))

# # print(pd.to_datetime('today') - start)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_10532/2758510078.py, line 11)

In [7]:
from sqlalchemy.types import Integer, Text, String, DateTime, Float

...

table_name = 'arguments'

model_args.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=True,
    index_label='index_label',
    chunksize=10000,
    dtype={
    'camis': Integer,
    'inspection_bin': String,
    'time_til': Integer,
    'event': Integer 
    }
)

2021-11-17 00:34:54,167 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("arguments")
2021-11-17 00:34:54,168 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-17 00:34:54,169 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("arguments")
2021-11-17 00:34:54,169 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-17 00:34:54,171 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-17 00:34:54,173 INFO sqlalchemy.engine.Engine 
CREATE TABLE arguments (
	index_label BIGINT, 
	time_til INTEGER, 
	event INTEGER, 
	inspection_bin VARCHAR, 
	camis INTEGER
)


2021-11-17 00:34:54,173 INFO sqlalchemy.engine.Engine [no key 0.00043s] ()
2021-11-17 00:34:54,184 INFO sqlalchemy.engine.Engine CREATE INDEX ix_arguments_index_label ON arguments (index_label)
2021-11-17 00:34:54,185 INFO sqlalchemy.engine.Engine [no key 0.00067s] ()
2021-11-17 00:34:54,195 INFO sqlalchemy.engine.Engine COMMIT
2021-11-17 00:34:54,201 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-17 00:34:54,235 INFO s

In [2]:
df['month'] = df['inspection_date'].dt.month
df['score'] = df['score'].astype(int)
df.dropna(subset=['latitude','longitude'], inplace=True)

df.reset_index(inplace=True, drop=True)


In [6]:
from sqlalchemy import create_engine

db_uri = 'sqlite:///site.db'

engine = create_engine(db_uri, echo=True)

In [10]:
from sqlalchemy.types import Integer, Text, String, DateTime, Float

...

table_name = 'restaurants'

df.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=True,
    index_label='index_label',
    chunksize=10000,
    dtype={
    'camis': Integer, 
    'dba': String, 
    'inspection_date': String, 
    'action': String, 
    'score': Integer, 
    'inspection_type': String, 
    'record_date': String, 
    'violation_description': String, 
    'latitude': Float, 
    'longitude': Float, 
    'cuisine_description': String, 
    'address': String, 
    'small': String, 
    'month': Integer
    }
)

2021-11-16 10:35:40,996 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("restaurants")
2021-11-16 10:35:40,997 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-16 10:35:40,999 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("restaurants")
2021-11-16 10:35:41,000 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-16 10:35:41,001 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-11-16 10:35:41,002 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-16 10:35:41,003 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("restaurants")
2021-11-16 10:35:41,004 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-16 10:35:41,006 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-11-16 10:35:41,007 INFO sqlalchemy.engine.Engine [raw sql] ('restaurants',)
2021-11-16 10:35:41,008 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_ke

In [14]:
df.to_csv('./static/df.csv')

In [40]:

from datetime import datetime
from flask import Flask, render_template, url_for, flash, redirect
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///site.db'
db = SQLAlchemy(app)

C:\Users\a\AppData\Local\Programs\Python\Python39\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [41]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [42]:
db.create_all()

In [43]:
df.to_sql('restaurants', con=engine)

In [31]:
query = 'baked'

In [44]:
for i in engine.execute("SELECT DISTINCT dba, address, camis FROM restaurants WHERE small LIKE '%{}%' ".format(query)).fetchall():
    print(i)

('Baked', '359 Van Brunt Street, Brooklyn', 41077320)
('Baked By Melissa', 'Nka Jfk International Airport, Queens', 50054957)
('Baked Cravings', '1673 Lexington Avenue, Manhattan', 50063001)
('Rubyzaar Baked', '20 Union Square, Manhattan', 50101038)


In [120]:
for i in engine.execute("SELECT * FROM users WHERE camis = 50063001").fetchone():
    print(i)

333
50063001
Baked Cravings
2017-08-09 00:00:00.000000
Violations were cited in the following area(s).
11.0
Pre-permit (Operational) / Initial Inspection
2021-11-02 00:00:00.000000
Food contact surface not properly washed, rinsed and sanitized after each use and following any activity when contamination may have occurred.
40.791465560612
-73.946832482249
Bakery Products/Desserts
1673 Lexington Avenue, Manhattan
baked cravings
8


In [121]:
lat = 40.791465560612
lon = -73.946832482249

highLat = lat + (.012837 / 2)
lowLat = lat - (.012837 / 2)
highLon = lon + (.016154 / 2)
lowLon = lon - (.016154 / 2)


# where=latitude<40.683375804313 AND latitude>40.670538804313 AND longitude>-74.021419875196 AND longitude<-74.005265875196

In [122]:
bolo = engine.execute("SELECT violation_description FROM users WHERE latitude < {} AND latitude > {} AND longitude < {} AND longitude > {} AND month = 9".format(highLat,lowLat,highLon,lowLon)).fetchall()



In [14]:
import datetime

datetime.datetime.now().month

11

In [135]:
pd.Series(bolo).value_counts().values / len(bolo)

array([0.14963504, 0.10583942, 0.09489051, 0.08394161, 0.0620438 ,
       0.0620438 , 0.05474453, 0.05109489, 0.03649635, 0.03649635,
       0.02554745, 0.02554745, 0.02554745, 0.02554745, 0.01824818,
       0.01824818, 0.01824818, 0.01824818, 0.01824818, 0.01094891,
       0.01094891, 0.01094891, 0.00729927, 0.00364964, 0.00364964,
       0.00364964, 0.00364964, 0.00364964, 0.00364964, 0.00364964,
       0.00364964])

In [119]:
for i in engine.execute("SELECT inspection_date, violation_description FROM users WHERE camis = 41077320 ORDER BY inspection_date DESC").fetchall():
    print(i)

('2020-01-29 00:00:00.000000', 'Mechanical or natural ventilation system not provided,  improperly installed, in disrepair and/or fails to prevent excessive build-up of grease, heat, steam condensation vapors, odors, smoke, and fumes.')
('2020-01-29 00:00:00.000000', 'Food not cooled by an approved method whereby the internal product temperature is reduced from 140º F to 70º F or less within 2  hours, and from 70º F to 41º F or less within 4 additional hours.')
('2019-08-30 00:00:00.000000', 'Proper sanitization not provided for utensil ware washing operation.')
('2019-08-30 00:00:00.000000', 'Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly maintained and/or not properly sealed, raised, spaced or movable to allow accessibility for cleaning on all sides, above and underneath the unit.')
('2019-08-30 00:00:00.000000', 'Food not protected from potential source of contamination during storage, preparation, transp

In [34]:
import pandas as pd
import datetime
from lifelines import CoxPHFitter
import numpy as np
import pickle
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, String, Float

start = datetime.datetime.now()
origin = datetime.datetime.now()

df = pd.read_csv('./static/DOHMH_New_York_City_Restaurant_Inspection_Results (4).csv')

df.columns = [x.lower() for x in df.columns]
df.columns = [x.replace(' ','_') for x in df.columns]


df = df[(df['inspection_type'].str.contains('Cycle')) | 
(df['inspection_type'].str.contains('Pre')) |
        (pd.isna(df['inspection_type']))]
df = df[['camis', 'dba', 'building', 'street', 'boro', 'inspection_date', 'action', 'score','inspection_type','record_date','violation_description',
        'latitude','longitude','cuisine_description','zipcode']]

df['inspection_date'] = df['inspection_date'].astype('datetime64[ns]')
df['record_date'] = df['record_date'].astype('datetime64[ns]')

df = df[df['inspection_date']!='1900-01-01']
df['zipcode'] = df['zipcode'].astype(str).replace(r'\.0', '', regex=True)
df['dba'] = df['dba'].astype(str).apply(lambda x: ' '.join(x.split()).title())
df['address'] = df['building'] + ' ' + df['street'] + ', ' + df['boro']
df['address'] = df['address'].astype(str).apply(lambda x: ' '.join(x.split()).title())
df['address'] = df['address'].str.replace('\s{2,}', ' ', regex=True)


df.drop(columns=['building','street','boro'], inplace=True)


small = df['dba'].apply(lambda x: x.lower())
small = small.str.replace('[^a-zA-Z0-9 ]', '', regex=True)
df['small'] = small


df['month'] = df['inspection_date'].dt.month
# df['score'] = df['score'].astype(int)
df.dropna(subset=['latitude','longitude'], inplace=True)


df = df.sort_values(['camis','inspection_date'], ascending=True)
df.reset_index(inplace=True, drop=True)

end = datetime.datetime.now() - start

print('finished preprocessing in {}'.format(end))

# end of Restaurants preprocessing

# sqlite push
start = datetime.datetime.now()


# db_uri = 'sqlite:///site.db'
# engine = create_engine(db_uri, echo=False)
# table_name = 'restaurants'

# df.to_sql(
#     table_name,
#     engine,
#     if_exists='replace',
#     index=True,
#     index_label='index_label',
#     chunksize=10000,
#     dtype={
#     'camis': Integer, 
#     'dba': String, 
#     'inspection_date': String, 
#     'action': String, 
#     'score': Integer, 
#     'inspection_type': String, 
#     'record_date': String, 
#     'violation_description': String, 
#     'latitude': Float, 
#     'longitude': Float, 
#     'cuisine_description': String, 
#     'address': String, 
#     'small': String, 
#     'month': Integer
#     }
# )

# end = datetime.datetime.now() - start

# print('finished sql push in {}'.format(end))


# preprocess for model fitting

start = datetime.datetime.now()


df['next_grade'] = df['inspection_date'].shift(-1)

df.loc[df.drop_duplicates("camis", keep='last').index,'next_grade'] = pd.NaT

df['time_til'] = df['next_grade'] - df['inspection_date']

df['time_til'] = df['time_til'].where(pd.notnull(df['time_til']), (df['record_date'] - df['inspection_date']))

df['event'] = df['next_grade'].apply(lambda x: 1 if pd.notnull(x) else 0)

df['time_til'] = df['time_til'].dt.days

# df = df[df['time_til'] < 800]
df.reset_index(drop=True, inplace=True)

inspection_bin = []

for i in range(len(df)):
    current_type = df.loc[i,'inspection_type']
    current_score = df.loc[i,'score']
    current_camis = df.loc[i,'camis']
    current_action = df.loc[i,'action']
    current_time = df.loc[i,'time_til']
    current_event = df.loc[i,'event']

    if (df.loc[i,'inspection_date'] < pd.Timestamp('2020-03-17')) & (current_event == 0):
        inspection_bin.append('covid')

    elif (df.loc[i,'inspection_date'] < pd.Timestamp('2020-03-17')) & (df.loc[i,'next_grade'] > pd.Timestamp('2021-07-19')):
        inspection_bin.append('covid')
    
    elif 'losed' in current_action:
        inspection_bin.append('was_closed')
        
    elif 're-open' in current_action:
        inspection_bin.append('re-opened')
        
    elif 'Cycle' in current_type:
        if 'Initial' in current_type:
            if (current_score < 14) & (current_event == 1) & (current_time < 300):
                inspection_bin.append('cyc_init_1')
            elif current_score < 14: # This event shouldn't be able to occur
                inspection_bin.append('cyc_init_0')
            elif current_score > 13:
                inspection_bin.append('cyc_init_1')
                
        elif 'Re-' in current_type:
            previous_score = df.loc[i-1,'score']
            previous_camis = df.loc[i-1,'camis']
            
            if current_camis == previous_camis:
                if previous_score < 14:
                    inspection_bin.append('cyc_re_0')
                elif previous_score > 13 and previous_score < 28:
                    inspection_bin.append('cyc_re_1')
                elif previous_score > 27:
                    inspection_bin.append('cyc_re_2')
            else:
                inspection_bin.append('missing_prior_cycle')
        else:
            inspection_bin.append('other_cycle')
            
            
    elif 'Pre-' in current_type:
        if 'Initial' in current_type:
            if (current_score < 14) & (current_event == 1) & (current_time < 300):
                inspection_bin.append('pre_init_1')
            elif current_score < 14:
                inspection_bin.append('pre_init_0')
            elif current_score > 13:
                inspection_bin.append('pre_init_1')
                
        elif 'Re-' in current_type:
            previous_camis = df.loc[i-1,'camis']
            previous_score = df.loc[i-1,'score']
            
            if current_camis == previous_camis:
                if previous_score < 14:
                    inspection_bin.append('pre_re_0')
                elif previous_score > 13 and previous_score < 28:
                    inspection_bin.append('pre_re_1')
                elif previous_score > 27:
                    inspection_bin.append('pre_re_2')
            else:
                inspection_bin.append('missing_prior_pre')
        else:
            inspection_bin.append('other_pre')
    else:
        inspection_bin.append('other')
        

inspection_bin = pd.Series(inspection_bin)

df['inspection_bin'] = inspection_bin


# fit proportional hazards model


cph = CoxPHFitter()
cph.fit(df[['time_til','event','inspection_bin']], duration_col='time_til', event_col='event', strata=['inspection_bin'])


filename = './static/todays_model.sav'
pickle.dump(cph, open(filename, 'wb'))

end = datetime.datetime.now() - start
print('finished model fit in {}'.format(end))

# build reference table for model args

start = datetime.datetime.now()

model_args = df.drop_duplicates('camis', keep="last")[['time_til','event','inspection_bin','camis']]
model_args.reset_index(drop=True, inplace=True)

table_name = 'arguments'

model_args.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=True,
    index_label='index_label',
    chunksize=10000,
    dtype={
    'camis': Integer,
    'inspection_bin': String,
    'time_til': Integer,
    'event': Integer 
    }
)


end = datetime.datetime.now() - start
print('model args done in {}'.format(end))

end = datetime.datetime.now() - origin
print('done done in {}'.format(end))


finished preprocessing in 0:00:05.826411
finished sql push in 0:00:08.164823
finished model fit in 0:00:39.367693
model args done in 0:00:00.252000
done done in 0:00:53.611928


In [43]:
model_args['inspection_bin'].value_counts()
model_args[model_args['inspection_bin']=='cyc_init_1']

,time_til,event,inspection_bin,camis
4,109,0,cyc_init_1,40356483
11,113,0,cyc_init_1,40361618
21,97,0,cyc_init_1,40363427
25,106,0,cyc_init_1,40363834
40,104,0,cyc_init_1,40364439
...,...,...,...,...
24956,47,0,cyc_init_1,50109072
24965,12,0,cyc_init_1,50109105
25140,82,0,cyc_init_1,50110934
25181,29,0,cyc_init_1,50111306


In [37]:
20067/len(model_args)

0.789076324171287

In [44]:
df[(df['inspection_date']<'2020-3-17') & 
    (df['event']==0)].iloc[70:90,:]

,camis,dba,inspection_date,action,score,inspection_type,record_date,violation_description,latitude,longitude,cuisine_description,zipcode,address,small,month,next_grade,time_til,event,inspection_bin
1695,40366487,Brennan & Carr,2020-02-11,Violations were cited in the following area(s).,2.0,Cycle Inspection / Re-inspection,2021-11-16,Non-food contact surface improperly constructe...,40.600386,-73.941893,American,11229,"3432 Nostrand Avenue, Brooklyn",brennan carr,2,NaT,644,0,covid
1745,40366543,Jardin De China Rest,2020-01-27,Violations were cited in the following area(s).,24.0,Cycle Inspection / Re-inspection,2021-11-16,Evidence of mice or live mice present in facil...,40.750755,-73.870360,Chinese,11368,"3737 Junction Boulevard, Queens",jardin de china rest,1,NaT,659,0,covid
1785,40366586,Shimons Kosher Pizza,2019-12-04,Violations were cited in the following area(s).,12.0,Cycle Inspection / Re-inspection,2021-11-16,Non-food contact surface improperly constructe...,40.727181,-73.822096,Jewish/Kosher,11367,"7124 Main Street, Queens",shimons kosher pizza,12,NaT,713,0,covid
1857,40366716,Shun Lee Palace Restaurant,2020-03-05,Violations were cited in the following area(s).,26.0,Cycle Inspection / Initial Inspection,2021-11-16,Pesticide use not in accordance with label or ...,40.759377,-73.969484,Chinese,10022,"155 East 55 Street, Manhattan",shun lee palace restaurant,3,NaT,621,0,covid
1883,40366742,Tamaqua,2020-01-07,Violations were cited in the following area(s).,30.0,Cycle Inspection / Re-inspection,2021-11-16,"Food contact surface not properly washed, rins...",40.594352,-73.930157,Bottled Beverages,11229,"84 Ebony Court, Brooklyn",tamaqua,1,NaT,679,0,covid
1908,40366812,Cara Mia,2019-11-07,Violations were cited in the following area(s).,10.0,Cycle Inspection / Re-inspection,2021-11-16,Facility not vermin proof. Harborage or condit...,40.730704,-73.743267,Italian,11427,"22020 Hillside Avenue, Queens",cara mia,11,NaT,740,0,covid
1943,40366928,Richmond County Country Club,2019-06-05,Violations were cited in the following area(s).,13.0,Cycle Inspection / Initial Inspection,2021-11-16,"Single service item reused, improperly stored,...",40.587936,-74.104073,American,10304,"135 Flagg Place, Staten Island",richmond county country club,6,NaT,895,0,covid
1967,40366929,Little Brown Jug,2018-08-28,Establishment Closed by DOHMH. Violations were...,40.0,Cycle Inspection / Initial Inspection,2021-11-16,Non-food contact surface improperly constructe...,40.700794,-73.826448,American,11418,"124-02 Jamaica Avenue, Queens",little brown jug,8,NaT,1176,0,covid
2005,40366961,Mitchell'S Restaurant,2019-08-17,Violations were cited in the following area(s).,13.0,Cycle Inspection / Initial Inspection,2021-11-16,Non-food contact surface improperly constructe...,40.678850,-73.968417,Soul Food,11238,"617A Vanderbilt Avenue, Brooklyn",mitchells restaurant,8,NaT,822,0,covid
2043,40367038,Mexico Lindo Restaurant,2020-01-16,Violations were cited in the following area(s).,9.0,Cycle Inspection / Initial Inspection,2021-11-16,Pesticide use not in accordance with label or ...,40.739644,-73.979615,Mexican,10010,"459 2 Avenue, Manhattan",mexico lindo restaurant,1,NaT,670,0,covid


In [1]:
df

NameError: name 'df' is not defined

In [3]:
cammy = 50103003

In [2]:
import pandas as pd
import datetime
from lifelines import CoxPHFitter
import numpy as np
import pickle
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, String, Float

start = datetime.datetime.now()
origin = datetime.datetime.now()

df = pd.read_csv('./static/DOHMH_New_York_City_Restaurant_Inspection_Results (4).csv')

df.columns = [x.lower() for x in df.columns]
df.columns = [x.replace(' ','_') for x in df.columns]


df = df[(df['inspection_type'].str.contains('Cycle')) | 
(df['inspection_type'].str.contains('Pre')) |
        (pd.isna(df['inspection_type']))]
df = df[['camis', 'dba', 'building', 'street', 'boro', 'inspection_date', 'action', 'score','inspection_type','record_date','violation_description',
        'latitude','longitude','cuisine_description','zipcode']]

df['inspection_date'] = df['inspection_date'].astype('datetime64[ns]')
df['record_date'] = df['record_date'].astype('datetime64[ns]')

df = df[df['inspection_date']!='1900-01-01']
df['zipcode'] = df['zipcode'].astype(str).replace(r'\.0', '', regex=True)
df['dba'] = df['dba'].astype(str).apply(lambda x: ' '.join(x.split()).title())

df['dba'] = df['dba'].replace(r'(\'S)', "'s", regex=True)


df['address'] = df['building'] + ' ' + df['street'] + ', ' + df['boro']
df['address'] = df['address'].astype(str).apply(lambda x: ' '.join(x.split()).title())
df['address'] = df['address'].str.replace('\s{2,}', ' ', regex=True)


df.drop(columns=['building','street','boro'], inplace=True)


small = df['dba'].apply(lambda x: x.lower())
small = small.str.replace('[^a-zA-Z0-9 ]', '', regex=True)
df['small'] = small


df['month'] = df['inspection_date'].dt.month
# df['score'] = df['score'].astype(int)
df.dropna(subset=['latitude','longitude'], inplace=True)


df = df.sort_values(['camis','inspection_date'], ascending=True)
df.reset_index(inplace=True, drop=True)

end = datetime.datetime.now() - start

print('finished preprocessing in {}'.format(end))

# end of Restaurants preprocessing

# sqlite push
start = datetime.datetime.now()


db_uri = 'sqlite:///site.db'
engine = create_engine(db_uri, echo=False)
table_name = 'restaurants'

df.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=True,
    index_label='index_label',
    chunksize=10000,
    dtype={
    'camis': Integer, 
    'dba': String, 
    'inspection_date': String, 
    'action': String, 
    'score': Integer, 
    'inspection_type': String, 
    'record_date': String, 
    'violation_description': String, 
    'latitude': Float, 
    'longitude': Float, 
    'cuisine_description': String, 
    'address': String, 
    'small': String, 
    'month': Integer
    }
)

end = datetime.datetime.now() - start

print('finished sql push in {}'.format(end))


# preprocess for model fitting

start = datetime.datetime.now()


df['next_grade'] = df['inspection_date'].shift(-1)

df.loc[df.drop_duplicates("camis", keep='last').index,'next_grade'] = pd.NaT

df['time_til'] = df['next_grade'] - df['inspection_date']

df['time_til'] = df['time_til'].where(pd.notnull(df['time_til']), (df['record_date'] - df['inspection_date']))

df['event'] = df['next_grade'].apply(lambda x: 1 if pd.notnull(x) else 0)

df['time_til'] = df['time_til'].dt.days

df = df[df['time_til'] < 1000]
df.reset_index(drop=True, inplace=True)

inspection_bin = []

for i in range(len(df)):
    current_type = df.loc[i,'inspection_type']
    current_score = df.loc[i,'score']
    current_camis = df.loc[i,'camis']
    current_action = df.loc[i,'action']
    current_time = df.loc[i,'time_til']
    current_event = df.loc[i,'event']

    if (df.loc[i,'inspection_date'] < pd.Timestamp('2020-03-17')) & (current_event == 0):
        inspection_bin.append('covid')

    elif (df.loc[i,'inspection_date'] < pd.Timestamp('2020-03-17')) & (df.loc[i,'next_grade'] > pd.Timestamp('2021-07-19')):
        inspection_bin.append('covid')
    
    elif 'losed' in current_action:
        inspection_bin.append('was_closed')
        
    elif 're-open' in current_action:
        inspection_bin.append('re-opened')
        
    elif 'Cycle' in current_type:
        if 'Initial' in current_type:
            if (current_score < 14) & (current_event == 1) & (current_time < 300):
                inspection_bin.append('cyc_init_1')
            elif current_score < 14: # This event shouldn't be able to occur
                inspection_bin.append('cyc_init_0')
            elif current_score > 13:
                inspection_bin.append('cyc_init_1')
                
        elif 'Re-' in current_type:
            previous_score = df.loc[i-1,'score']
            previous_camis = df.loc[i-1,'camis']
            
            if current_camis == previous_camis:
                if previous_score < 14:
                    inspection_bin.append('cyc_re_0')
                elif previous_score > 13 and previous_score < 28:
                    inspection_bin.append('cyc_re_1')
                elif previous_score > 27:
                    inspection_bin.append('cyc_re_2')
            else:
                inspection_bin.append('missing_prior_cycle')
        else:
            inspection_bin.append('other_cycle')
            
            
    elif 'Pre-' in current_type:
        if 'Initial' in current_type:
            if (current_score < 14) & (current_event == 1) & (current_time < 300):
                inspection_bin.append('pre_init_1')
            elif current_score < 14:
                inspection_bin.append('pre_init_0')
            elif current_score > 13:
                inspection_bin.append('pre_init_1')
                
        elif 'Re-' in current_type:
            previous_camis = df.loc[i-1,'camis']
            previous_score = df.loc[i-1,'score']
            
            if current_camis == previous_camis:
                if previous_score < 14:
                    inspection_bin.append('pre_re_0')
                elif previous_score > 13 and previous_score < 28:
                    inspection_bin.append('pre_re_1')
                elif previous_score > 27:
                    inspection_bin.append('pre_re_2')
            else:
                inspection_bin.append('missing_prior_pre')
        else:
            inspection_bin.append('other_pre')
    else:
        inspection_bin.append('other')
        

inspection_bin = pd.Series(inspection_bin)

df['inspection_bin'] = inspection_bin


# fit proportional hazards model


cph = CoxPHFitter()
cph.fit(df[['time_til','event','inspection_bin']], duration_col='time_til', event_col='event', strata=['inspection_bin'])


filename = './static/todays_model.sav'
pickle.dump(cph, open(filename, 'wb'))

end = datetime.datetime.now() - start
print('finished model fit in {}'.format(end))

# build reference table for model args

start = datetime.datetime.now()

model_args = df.drop_duplicates('camis', keep="last")[['time_til','event','inspection_bin','camis']]
model_args.reset_index(drop=True, inplace=True)

table_name = 'arguments'

model_args.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=True,
    index_label='index_label',
    chunksize=10000,
    dtype={
    'camis': Integer,
    'inspection_bin': String,
    'time_til': Integer,
    'event': Integer 
    }
)


end = datetime.datetime.now() - start
print('model args done in {}'.format(end))

end = datetime.datetime.now() - origin
print('done done in {}'.format(end))


finished preprocessing in 0:00:06.459719
finished sql push in 0:00:08.630049
finished model fit in 0:00:42.476649
model args done in 0:00:00.206639
done done in 0:00:57.775059


In [6]:
model_args[model_args['camis']==50103003]

,time_til,event,inspection_bin,camis


In [7]:
df[df['camis']==50103003]

,camis,dba,inspection_date,action,score,inspection_type,record_date,violation_description,latitude,longitude,cuisine_description,zipcode,address,small,month,next_grade,time_til,event,inspection_bin
